# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [111]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
import numpy as np

In [112]:
# Read the Mapbox API key
load_dotenv("keys.env")
map_box_api = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [113]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
grouped_df = sfo_data.groupby("year").mean()
neighborhood_df = sfo_data.groupby(["year", "neighborhood"]).mean()
neighborhood_df = neighborhood_df.reset_index()
neighborhood_df_top_ten = neighborhood_df.groupby(["neighborhood"]).mean()
neighborhood_df_top_ten = neighborhood_df_top_ten.sort_values("sale_price_sqr_foot", ascending=False).reset_index().loc[0:9]
coord_path = Path("Data/neighborhoods_coordinates.csv")
coordinates = pd.read_csv(coord_path)
neighborhood_df_grouped = neighborhood_df.groupby("neighborhood").mean().reset_index().drop("year", axis=1)
neighborhood_df_grouped = pd.concat([neighborhood_df_grouped, coordinates], axis = "columns", join = "inner")
neighborhood_df_grouped.drop(["neighborhood"],axis = 1)
neighborhood_df_grouped = neighborhood_df_grouped[["Neighborhood", "Lat", "Lon", "sale_price_sqr_foot", "housing_units", "gross_rent"]]
neighborhood_yearly_df = neighborhood_df.loc[neighborhood_df['neighborhood'].isin(neighborhood_df_top_ten["neighborhood"])]
neighborhood_yearly_df = neighborhood_yearly_df.groupby(["neighborhood", "year"]).mean().reset_index()

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [114]:
# Define Panel Visualization Functions
def housing_units_per_year():
    mean = np.mean(grouped_df["housing_units"])
    stdev = np.std(grouped_df["housing_units"])
    minimum = np.min(grouped_df["housing_units"])
    maximum = np.max(grouped_df["housing_units"])
    plot = grouped_df.hvplot(kind = "bar", y="housing_units", ylim = ((minimum-stdev),(maximum+stdev))).opts(yformatter="%.0f")
    panel = pn.panel(plot)
    return panel


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    plot = grouped_df.hvplot(y = ["gross_rent"], color="red")
    panel = pn.panel(plot)
    return panel


def average_sales_price():
    """Average Sales Price Per Year."""
    
    plot = grouped_df.hvplot(y = ["sale_price_sqr_foot"], color="green")
    panel = pn.panel(plot)
    return panel



def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    plot = neighborhood_df.hvplot(x= "year",y="sale_price_sqr_foot", groupby="neighborhood", title = None, height=400)
    panel = pn.panel(plot)
    return panel


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    plot = neighborhood_df_top_ten.hvplot(kind="bar", x = "neighborhood", y="sale_price_sqr_foot", rot=45, height=400)
    panel = pn.panel(plot)
    return panel


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    plot = neighborhood_df.hvplot(kind ="bar", x= "year",y=["sale_price_sqr_foot", "gross_rent"], groupby="neighborhood", rot=90, height=400)
    panel = pn.panel(plot)
    return panel
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    plot = px.parallel_coordinates(
    neighborhood_df_top_ten,
    dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
    color="sale_price_sqr_foot",
    color_continuous_scale=px.colors.sequential.Inferno,
    width = 1000,
    height = 400
    )
    panel = pn.panel(plot)
    return panel



def parallel_categories():
    """Parallel Categories Plot."""
    
    plot = px.parallel_categories(
    neighborhood_df_top_ten,
    dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
    color="sale_price_sqr_foot",
    color_continuous_scale=px.colors.sequential.Inferno,
    width = 1000,
    height = 400
    )
    
    panel = pn.panel(plot)
    return panel


def neighborhood_map():
    """Neighborhood Map."""

    px.set_mapbox_access_token(map_box_api)

    # Create a scatter mapbox to analyze neighborhood info
    map_plot = px.scatter_mapbox(
        neighborhood_df_grouped,
        lat="Lat",
        lon="Lon",
        size="sale_price_sqr_foot",
        color="Neighborhood",
        color_discrete_sequence=px.colors.sequential.Plasma_r,
        zoom=9,
        title= "San Fransisco",
        width = 1000,
        height = 800
    )

    panel = pn.panel(map_plot)
    return panel


def sunburst():
    """Sunburst Plot."""
    
    plot = px.sunburst(neighborhood_yearly_df, path = ["year", "neighborhood"], values="sale_price_sqr_foot", color="gross_rent", title="Cost Analysis of Top 10 Most Expensive SF Neighborhoods",color_continuous_scale='blues', height=700)
    panel = pn.panel(plot)
    return panel

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [115]:
# Create a Title for the Dashboard
dashboard_title = "San Francisco Housing Market"


# Create a tab layout for the dashboard
intro = pn.Column("## San Fransisco has been one of the hottest real estate markets in the Unites States since the turn of the millenia. See why investors from all around the world are looking to get into the action!", neighborhood_map())
yearly = pn.Row(pn.Column("# Housing Unit Growth", housing_units_per_year()), pn.Column("# Average Gross Rent Growth", average_gross_rent()), pn.Column("# Average Price Growth", average_sales_price()))
hoods = pn.Row(pn.Column("#Average Sales Price",average_price_by_neighborhood()),pn.Column("# The Most Expensive Neighborhoods in SF",top_most_expensive_neighborhoods()),pn.Column("# A Deeper Look into the Neighborhoods",most_expensive_neighborhoods_rent_sales()))
advanced = pn.Column("# San Fransisco Interactive Analysis", pn.Row(parallel_coordinates(),parallel_categories()))
sunburst = pn.Column("# Summary of San Fransisco Housing Market", sunburst())

# Create the dashboard
dash = pn.Tabs(("Introduction", intro), ("Yearly Analysis", yearly),("Neighborhood Analysis", hoods), ("Advanced Analysis", advanced), ("Summary", sunburst))

## Serve the Panel Dashboard

In [116]:
# Serve the# dashboard
dash.show()

Launching server at http://localhost:58758


# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()